# Основи мереж зі Scapy

## 1 Загальні налаштування Scapy 

### 1.1 Встановлення Scapy

**Вимоги** - має бути встановлені `python3` і пакетний менеджер `pip`

Документація по встановленню: [[https://scapy.readthedocs.io/en/latest/installation.html]]

Встановлення Scapy:  
> В залежності від ОС може потребувати `pip3` замість `pip`  

Команда командного рядка:
```  
pip install scapy  
```
або засобами Python прямо в блокноті:

In [ ]:
!pip install scapy

### 1.2 Імпорт Scapy

In [2]:
from scapy.all import *

### 1.3 Особливості операційних систем

##### Windows
Після імпорту бібліотеки у випадку попередження 
```
WARNING! No libpcap provider available! pcap won`t be used
``` 
слід також встановити програму-аналізатор мережних пакетів Wireshark з офіційної сторінки https://www.wireshark.org
> Програма знадобиться як в рамках даного курсу, так і з інших мережних дисциплін

##### Linux
В ОС Linux Scapy потребує прав адміністратора для доступу до сокета і роботою з пакетами. У вікні термінала запускається команда з підтвердженням паролем
```
sudo jupyter notebook --allow-root
```
Після запуску Jupyter-сервера відображається **одноразове** посилання для роботи у браузері, подібне до `http://127.0.0.1:8888/tree?token=0c86a75d0aa67fc4cb412ee587b990806b3095b63e9d1a6f`

### 1.4 Інтеграція з додатками

1. Робота в браузері - відкрити посилання і знайти файл блокноту в домашньому каталозі
2. Робота в Intellij IDEA / PyCharm: `Tools -> Add Jupyter Connection -> Connect to Jupyter server using URL` (не запрацювало)
3. Робота в Visual Studio Code: `Select Kernel -> Select Another Kernel -> Existing Jupyter Server -> Enter the URL`


## 2 Основи Scapy

З мануалу https://scapy.readthedocs.io/en/latest/usage.html

### 2.1 Створення пакету і стекування рівнів

##### Створення пакету

У Scapy кожен мережний рівень є класом Python.

Створення пакету ІР та приклади інших протоколів з певними параметрами:

In [1]:
from scapy.all import *

ip  = IP(ttl=10, dst="192.168.1.1")
ip

<IP  ttl=10 dst=192.168.1.1 |>

In [7]:
i = IP()
t = TCP(dport=80)
e = Ether()

##### Стекування протоколів

**Оператор '/'** - поєднання протоколів різного рівня \
Використовується для зв'язування шарів. Розміщення сегменту TCP поверх IP і призначення його змінній пакету, а потім накладання його поверх Ethernet.

Вивід відображає узагальнену інформацію пакета. Scapy автоматично заповнив тип Ethernet, а також поле IP-протоколу.

In [9]:
packet = IP(ttl=10, dst="192.168.1.1")/TCP(dport=80)
eth = Ether()/packet
eth

<Ether  type=IPv4 |<IP  frag=0 ttl=10 proto=tcp dst=192.168.1.1 |<TCP  dport=http |>>>

##### Створення пакету з заданим полем

In [3]:
Ether()/IP(dst="8.8.8.8")/TCP()

<Ether  type=IPv4 |<IP  frag=0 proto=tcp dst=8.8.8.8 |<TCP  |>>>

##### Створення пакету по доменному імені адресата

In [4]:
p = Ether()/IP(dst="www.secdev.org")/TCP()
p

<Ether  type=IPv4 |<IP  frag=0 proto=tcp dst=Net("www.secdev.org/32") |<TCP  |>>>

### 2.2 Відображення інформації

##### Основні команди (таблиця)

Команда|Ефект
-|-
raw (pkt)|зібрати пакет
hexdump(pkt)|отримання шістнадцяткового дампу
ls (pkt)|отримання списку значень полів
pkt.summary()|однорядкове резюме
pkt.show()|розгорнутий вигляд пакета
pkt.show2()|те саме на зібраному пакеті (наприклад, ще обчислюється контрольна сума)
pkt.sprintf()|Форматування рядка зі значеннями полів
pkt.decode_payload_as()|змінює спосіб декодування корисного навантаження
pkt.psdump()| малює діаграму PostScript із поясненим розбором
pkt.pdfdump()| малює PDF із поясненим розбором 
pkt.command() | повертає команду Scapy, яка може генерувати пакет


##### Відображення полей/атрибутів протокола (ls)

**ls()** - функція, що відображає поля протоколу. Без аргументів функція відображає всі протоколи, що підтримує. 

`verbose=True` - відображення всіх можливих значень переліку

Результати виводу відображають поля мережного та інших рівнів і імена атрибутів, за якими до них можна звернутися.

Приклади:

In [10]:
ls(IP, verbose=True)

version    : BitField  (4 bits)                  = ('4')
ihl        : BitField  (4 bits)                  = ('None')
tos        : XByteField                          = ('0')
len        : ShortField                          = ('None')
id         : ShortField                          = ('1')
flags      : FlagsField                          = ('<Flag 0 ()>')
               MF, DF, evil
frag       : BitField  (13 bits)                 = ('0')
ttl        : ByteField                           = ('64')
proto      : ByteEnumField                       = ('0')
chksum     : XShortField                         = ('None')
src        : SourceIPField                       = ('None')
dst        : DestIPField                         = ('None')
options    : PacketListField                     = ('[]')


In [ ]:
ls(Ether)

dst        : DestMACField                        = ('None')
src        : SourceMACField                      = ('None')
type       : XShortEnumField                     = ('36864')


In [ ]:
ls(TCP)

sport      : ShortEnumField                      = ('20')
dport      : ShortEnumField                      = ('80')
seq        : IntField                            = ('0')
ack        : IntField                            = ('0')
dataofs    : BitField  (4 bits)                  = ('None')
reserved   : BitField  (3 bits)                  = ('0')
flags      : FlagsField                          = ('<Flag 2 (S)>')
window     : ShortField                          = ('8192')
chksum     : XShortField                         = ('None')
urgptr     : ShortField                          = ('0')
options    : TCPOptionsField                     = ("b''")


In [ ]:
ls(UDP)

sport      : ShortEnumField                      = ('53')
dport      : ShortEnumField                      = ('53')
len        : ShortField                          = ('None')
chksum     : XShortField                         = ('None')


##### Стисле резюме (summary)

**summary()** - функція відображення стислої інформації про об'єкт

In [5]:
p = Ether()/IP(dst="8.8.8.8")/TCP()
p.summary()


'Ether / IP / TCP 192.168.31.195:ftp_data > 8.8.8.8:http S'

##### Доступ до протоколів і властивостей пакету

Стек протоколів сформованого кадру/пакету зберігається у вигляді словника з ключами - класами протоколів

In [19]:
print(p.src)        # Ether
print(p[Ether].src) # Ether
print(p[IP].dst)    # IP


f8:ac:65:88:a7:2a
f8:ac:65:88:a7:2a
8.8.8.8


In [ ]:
ls(p[Ether])
ls(p[IP])
ls(p[TCP])

dst        : DestMACField                        = '9c:9d:7e:49:4a:8e' ('None')
src        : SourceMACField                      = 'f8:ac:65:88:a7:2a' ('None')
type       : XShortEnumField                     = 2048            ('36864')
--
version    : BitField  (4 bits)                  = 4               ('4')
ihl        : BitField  (4 bits)                  = None            ('None')
tos        : XByteField                          = 0               ('0')
len        : ShortField                          = None            ('None')
id         : ShortField                          = 1               ('1')
flags      : FlagsField                          = <Flag 0 ()>     ('<Flag 0 ()>')
frag       : BitField  (13 bits)                 = 0               ('0')
ttl        : ByteField                           = 64              ('64')
proto      : ByteEnumField                       = 6               ('0')
chksum     : XShortField                         = None            ('None')
src       

##### Форматований вивід інформації

**sprintf()** - Форматований вивід інформації

In [ ]:
print(p.sprintf("%Ether.src% > %Ether.dst%\n%IP.src% > %IP.dst%"))

f8:ac:65:88:a7:2a > 9c:9d:7e:49:4a:8e
192.168.31.195 > 8.8.8.8


In [ ]:
print(p.dst)  # first layer that has an src field, here Ether
print(p[IP].src)  # explicitly access the src field of the IP layer

# sprintf() is a useful method to display fields
print(p.sprintf("%Ether.src% > %Ether.dst%\n%IP.src% > %IP.dst%"))

9c:9d:7e:49:4a:8e
192.168.31.195
f8:ac:65:88:a7:2a > 9c:9d:7e:49:4a:8e
192.168.31.195 > Net("www.secdev.org/32")


### 2.3 Набори пакетів

##### Дамп і РСАР-файли

Зчитування pcap файлів, сформованих программами по захопленню мережного трафіку (Wireshark)

In [13]:
a=rdpcap("/home/oleg/wrshrk_test.pcap")
a

<wrshrk_test.pcap: TCP:93 UDP:1302 ICMP:0 Other:5>

##### Генерування набору пакетів

Генерування ІР-пакетів по масі підмережі

In [23]:
a=IP(dst="www.slashdot.org/30")
a

<IP  dst=Net("www.slashdot.org/30") |>

In [24]:
[p for p in a]

[<IP  dst=172.64.151.192 |>,
 <IP  dst=172.64.151.193 |>,
 <IP  dst=172.64.151.194 |>,
 <IP  dst=172.64.151.195 |>]

В Scapy існує можливість створювати послідовність пакетів з послідовною зміною заданого поля `(m, n)`
Наприклад, зміна значення поля TTL від 1 до 5 імітує traceroute.

In [ ]:
[p for p in IP(ttl=(1,5))/ICMP()]

[<IP  frag=0 ttl=1 proto=icmp |<ICMP  |>>,
 <IP  frag=0 ttl=2 proto=icmp |<ICMP  |>>,
 <IP  frag=0 ttl=3 proto=icmp |<ICMP  |>>,
 <IP  frag=0 ttl=4 proto=icmp |<ICMP  |>>,
 <IP  frag=0 ttl=5 proto=icmp |<ICMP  |>>]

In [25]:
b=IP(ttl=[1,2,(5,9)])
[p for p in b]

<IP  ttl=[1, 2, (5, 9)] |>

Послідовне генерування пакетів по 4м ір-адресам і двом портам послідовно

In [27]:
c=TCP(dport=[80,443])
[p for p in a/c]

[<IP  frag=0 proto=tcp dst=172.64.151.192 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.192 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.193 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.193 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.194 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.194 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.195 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=172.64.151.195 |<TCP  dport=https |>>]

##### Функції відображення для набору пакетів

Команда|Ефект
-|-
summary()|відображає список підсумків кожного пакета
nsummary()|те саме, що й попередній, із номером пакета
conversations()| відображає графік діалогів
show()|відображає бажане представлення (зазвичай nsummary())
filter()|повертає список пакетів, відфільтрований лямбда-функцією
hexdump()|повертає 16-ковий дамп усіх пакетів
hexraw()|повертає шістнадцятковий дамп рівня Raw усіх пакетів
padding()|повертає шістнадцятковий дамп пакетів із заповненням
nzpadding()|повертає шістнадцятковий дамп пакетів із ненульовим доповненням
plot()|будує графік лямбда-функції, застосованої до списку пакетів
make_table()|відображає таблицю відповідно до лямбда-функції

### 2.4 Відправка пакетів

##### Функції send() i sendp()

Функції відправки пакетів:
- **send()** надсилатиме пакети на рівні 3 - оброблятиме маршрутизацію та генерує каадр рівня 2 за нас.
- **sendp()** працює на рівні 2 - необхідно вибрати правильний інтерфейс і правильний протокол канального рівня

send() і sendp() також повернуть список надісланих пакетів, якщо передано як параметр `return_packets=True`

In [28]:
send(IP(dst="1.2.3.4")/ICMP())


Sent 1 packets..


In [30]:
sendp(Ether()/IP(dst="1.2.3.4",ttl=(1,4)), iface="wlo1")


...
Sent 4 packets.
.

In [31]:
sendp("I'm travelling on Ethernet", iface="wlo1", loop=1, inter=0.2)

................................................................................................................................................................
Sent 160 packets.


In [32]:
send(IP(dst='127.0.0.1'), return_packets=True)


Sent 1 packets.
.

<PacketList: TCP:0 UDP:0 ICMP:0 Other:1>

### 2.5 Відправка і отримання пакетів

##### Функції сімейства sr*

- **Функція sr()** призначена для надсилання пакетів і отримання
відповідей. Функція повертає пари пакетів з відповідями, а також пакети без відповіді.
- **Функція sr1()** - варіант, який повертає лише один пакет, який відповів на надісланий пакет (або набір пакетів). Пакети мають бути пакетами рівня 3 (IP, ARP тощо).
- **Функція srp()** робить те саме для пакетів рівня 2 (Ethernet, 802.3). Якщо відповіді немає, після закінчення часу
очікування натомість буде призначено значення None.
- **Функція srp1()** - аналогічно повертає один пакет відповіді для пакету рівня 2.

##### Приклад запитів DNS

Приклад використання протокола DNS для отримання IPv4-адреси www.example.com.

In [33]:
p = sr1(IP(dst="www.slashdot.org")/ICMP()/"XXXXXXXXXXX")

Begin emission:
Finished sending 1 packets.
.*
Received 2 packets, got 1 answers, remaining 0 packets


DNS-запит (rd = recursion desired):

In [34]:
sr1(IP(dst="192.168.31.1")/UDP()/DNS(rd=1,qd=DNSQR(qname="www.slashdot.org")))

Begin emission:
Finished sending 1 packets.
.*
Received 2 packets, got 1 answers, remaining 0 packets


<IP  version=4 ihl=5 tos=0x0 len=143 id=39351 flags=DF frag=0 ttl=64 proto=udp chksum=0xe091 src=192.168.31.1 dst=192.168.31.195 |<UDP  sport=domain dport=domain len=123 chksum=0xd0c6 |<DNS  id=0 qr=1 opcode=QUERY aa=0 tc=0 rd=1 ra=1 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=3 nscount=0 arcount=0 qd=<DNSQR  qname='www.slashdot.org.' qtype=A qclass=IN |> an=<DNSRR  rrname='www.slashdot.org.' type=CNAME rclass=IN ttl=29 rdlen=37 rdata='www.slashdot.org.cdn.cloudflare.net.' |<DNSRR  rrname='www.slashdot.org.cdn.cloudflare.net.' type=A rclass=IN ttl=125 rdlen=4 rdata=172.64.151.192 |<DNSRR  rrname='www.slashdot.org.cdn.cloudflare.net.' type=A rclass=IN ttl=125 rdlen=4 rdata=104.18.36.64 |>>> ns=None ar=None |>>>

In [5]:
p = sr1(IP(dst="8.8.8.8")/UDP()/DNS(qd=DNSQR()))
p[DNS].an

Begin emission:
Finished sending 1 packets.
*
Received 1 packets, got 1 answers, remaining 0 packets


<DNSRR  rrname='www.example.com.' type=A rclass=IN ttl=1111 rdlen=4 rdata=93.184.216.34 |>

##### Обробка пакетів з відповідю та без

Особливістю Scapy є те, що сімейство функцій sr* повертає два списки:
- список пар (пакет надіслано, відповідь)
- список пакетів без відповіді.

Ці два елементи є списками, але вони обгорнуті об’єктом, щоб краще представити їх і надати їм деякі методи, які виконують найбільш часто потрібні дії:

In [41]:
sr(IP(dst="192.168.31.1")/TCP(dport=[21,22]))

Begin emission:
Finished sending 2 packets.
**
Received 2 packets, got 2 answers, remaining 0 packets


(<Results: TCP:2 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [42]:
ans, unans = _
ans.summary()

IP / TCP 192.168.31.195:ftp_data > 192.168.31.1:ssh S ==> IP / TCP 192.168.31.1:ssh > 192.168.31.195:ftp_data RA
IP / TCP 192.168.31.195:ftp_data > 192.168.31.1:ftp S ==> IP / TCP 192.168.31.1:ftp > 192.168.31.195:ftp_data RA


##### Запити з додатковими параметрами

Параметри:
- inter - проміжок часу (у секундах) між двома пакетами.
- retry - кількість повторних спроб, якщо не було відповіді (-2 - двічі)
- timeout визначає час очікування після надсилання останнього пакета.

In [43]:
sr(IP(dst="172.20.29.5/30")/TCP(dport=[21,22,23]),inter=0.5,retry=-2,timeout=1)

Begin emission:
Finished sending 12 packets.
.....................................Begin emission:
Finished sending 12 packets.
***....................***.Begin emission:
Finished sending 6 packets.
.................Begin emission:
Finished sending 6 packets.
.....
Received 86 packets, got 6 answers, remaining 6 packets


(<Results: TCP:0 UDP:0 ICMP:6 Other:0>,
 <Unanswered: TCP:6 UDP:0 ICMP:0 Other:0>)

##### Обробка окремих пакетів з набору і запис у дамп-файл

In [14]:
r, u = srp(Ether()/IP(dst="8.8.8.8", ttl=(5,10))/UDP()/DNS(rd=1, qd=DNSQR(qname="www.example.com")))
r, u

Begin emission:
Finished sending 6 packets.
******
Received 6 packets, got 6 answers, remaining 0 packets


(<Results: TCP:0 UDP:2 ICMP:4 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [10]:
print(r[0][0].summary())  # the packet sent
print(r[0][1].summary())  # the answer received

# Access the ICMP layer. Scapy received a time-exceeded error message
r[0][1][ICMP]

Ether / IP / UDP / DNS Qry "b'www.example.com.'" 
Ether / IP / ICMP 108.170.248.138 > 192.168.31.195 time-exceeded ttl-zero-during-transit / IPerror / UDPerror


<ICMP  type=time-exceeded code=ttl-zero-during-transit chksum=0x3012 reserved=0 length=0 unused=0 |<IPerror  version=4 ihl=5 tos=0x80 len=61 id=1 flags= frag=0 ttl=1 proto=udp chksum=0xc8b4 src=192.168.31.195 dst=8.8.8.8 |<UDPerror  sport=domain dport=domain len=41 chksum=0xc45a |>>>

In [11]:
wrpcap("scapy.pcap", r)

pcap_p = rdpcap("scapy.pcap")
pcap_p[0]

<Ether  dst=9c:9d:7e:49:4a:8e src=f8:ac:65:88:a7:2a type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=61 id=1 flags= frag=0 ttl=5 proto=udp chksum=0xc534 src=192.168.31.195 dst=8.8.8.8 |<UDP  sport=domain dport=domain len=41 chksum=0xc45a |<DNS  id=0 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='www.example.com.' qtype=A qclass=IN |> an=None ns=None ar=None |>>>>

### 2.6 Запити ping i traceroute

##### Ping

**Ping** (Packet Inter-Network Groper) - програма, призначена для перевірки з'єднань в мережах на основі TCP/IP. Вона відправляє запити протоколу ICMP зазначеному вузлу мережі й фіксує відповіді (Echo-Reply).

In [3]:
!ping -c5 8.8.8.8
!ping -c4 google.com 

PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=119 time=25.5 ms


64 bytes from 8.8.8.8: icmp_seq=2 ttl=119 time=25.2 ms
64 bytes from 8.8.8.8: icmp_seq=3 ttl=119 time=25.6 ms
64 bytes from 8.8.8.8: icmp_seq=4 ttl=119 time=22.8 ms
64 bytes from 8.8.8.8: icmp_seq=5 ttl=119 time=25.2 ms

--- 8.8.8.8 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 22.829/24.880/25.587/1.036 ms
PING google.com (142.250.203.142) 56(84) bytes of data.
64 bytes from waw07s06-in-f14.1e100.net (142.250.203.142): icmp_seq=1 ttl=119 time=24.6 ms
64 bytes from waw07s06-in-f14.1e100.net (142.250.203.142): icmp_seq=2 ttl=119 time=25.5 ms
64 bytes from waw07s06-in-f14.1e100.net (142.250.203.142): icmp_seq=3 ttl=119 time=25.7 ms
64 bytes from waw07s06-in-f14.1e100.net (142.250.203.142): icmp_seq=4 ttl=119 time=27.5 ms

--- google.com ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3005ms
rtt min/avg/max/mdev = 24.603/25.831/27.491/1.046 ms


Ping засобами Scapy

In [4]:
from scapy.all import *

ping = [IP(dst="8.8.8.8")/ICMP() for p in range(5)]

ans, unans = sr(ping, verbose=0)
ans.summary( lambda s,r : r.sprintf("%IP.src% is alive"))

8.8.8.8 is alive
8.8.8.8 is alive
8.8.8.8 is alive
8.8.8.8 is alive
8.8.8.8 is alive


In [5]:
[print(f"from {p.answer[IP].src}: icmp_seq={i+1} ttl={p.answer[IP].ttl}") for i, p in enumerate(ans)]


from 8.8.8.8: icmp_seq=1 ttl=119
from 8.8.8.8: icmp_seq=2 ttl=119
from 8.8.8.8: icmp_seq=3 ttl=119
from 8.8.8.8: icmp_seq=4 ttl=119
from 8.8.8.8: icmp_seq=5 ttl=119


[None, None, None, None, None]

In [6]:
print(ans[0][0])
print(ans[0][1])
ls(ans[0][0])

IP / ICMP 192.168.31.195 > 8.8.8.8 echo-request 0
IP / ICMP 8.8.8.8 > 192.168.31.195 echo-reply 0
version    : BitField  (4 bits)                  = 4               ('4')
ihl        : BitField  (4 bits)                  = None            ('None')
tos        : XByteField                          = 0               ('0')
len        : ShortField                          = None            ('None')
id         : ShortField                          = 1               ('1')
flags      : FlagsField                          = <Flag 0 ()>     ('<Flag 0 ()>')
frag       : BitField  (13 bits)                 = 0               ('0')
ttl        : ByteField                           = 64              ('64')
proto      : ByteEnumField                       = 1               ('0')
chksum     : XShortField                         = None            ('None')
src        : SourceIPField                       = '192.168.31.195' ('None')
dst        : DestIPField                         = '8.8.8.8'       ('None')

##### Traceroute

Traceroute (tracert в ОС Windows) — службова програма, призначена для визначення маршрутів слідування даних у мережах TCP/IP.

In [7]:
!traceroute www.google.co.uk

traceroute to www.google.co.uk (142.250.203.195), 30 hops max, 60 byte packets
 1  XiaoQiang (192.168.31.1)  2.210 ms  5.684 ms  5.672 ms
 2  * * *
 3  10.12.202.9 (10.12.202.9)  9.633 ms  9.622 ms  9.611 ms
 4  226.3.86.109.triolan.net (109.86.3.226)  11.112 ms  11.102 ms  11.093 ms
 5  108.170.248.155 (108.170.248.155)  11.091 ms  11.082 ms  11.072 ms
 6  142.251.242.35 (142.251.242.35)  25.468 ms 72.14.239.111 (72.14.239.111)  10.551 ms 142.251.242.35 (142.251.242.35)  26.072 ms
 7  142.250.46.55 (142.250.46.55)  26.044 ms 142.251.242.37 (142.251.242.37)  24.458 ms 142.251.242.41 (142.251.242.41)  40.906 ms
 8  209.85.250.175 (209.85.250.175)  25.259 ms  24.895 ms 108.170.250.209 (108.170.250.209)  25.669 ms
 9  209.85.250.175 (209.85.250.175)  26.498 ms 209.85.252.109 (209.85.252.109)  26.487 ms  26.478 ms
10  waw02s22-in-f3.1e100.net (142.250.203.195)  25.629 ms  25.257 ms  24.976 ms


In [8]:
ans, unans = sr(IP(dst="www.google.co.uk", ttl=(1,25),id=RandShort())/TCP(flags=0x2))
ans.summary(lambda s,r: r.sprintf("%IP.src%"))

Begin emission:
Finished sending 25 packets.
.**********................................................
Received 59 packets, got 10 answers, remaining 15 packets
192.168.31.1
10.12.202.9
109.86.3.226
108.170.248.138
72.14.239.111
142.251.242.37
209.85.250.175
108.170.250.193
142.250.203.195
142.250.203.195


In [9]:
r2, unans = traceroute(["www.google.co.uk"],maxttl=25)

Begin emission:
Finished sending 25 packets.
************************
Received 24 packets, got 24 answers, remaining 1 packets
   142.250.203.195:tcp80 
1  192.168.31.1    11    
3  10.12.202.9     11    
4  109.86.3.226    11    
5  108.170.248.138 11    
6  142.251.242.35  11    
7  142.251.242.41  11    
8  209.85.250.175  11    
9  142.250.203.195 SA    
10 142.250.203.195 SA    
11 142.250.203.195 SA    
12 142.250.203.195 SA    
13 142.250.203.195 SA    
14 142.250.203.195 SA    
15 142.250.203.195 SA    
16 142.250.203.195 SA    
17 142.250.203.195 SA    
18 142.250.203.195 SA    
19 142.250.203.195 SA    
20 142.250.203.195 SA    
21 142.250.203.195 SA    
22 142.250.203.195 SA    
23 142.250.203.195 SA    
24 142.250.203.195 SA    
25 142.250.203.195 SA    


In [13]:
for snd,rcv in ans:
    print( snd.ttl, rcv.src, isinstance(rcv.payload, TCP))

4 109.86.3.226 False
5 72.14.239.111 False
6 108.170.248.138 False
7 108.170.250.193 False
8 142.251.242.37 False
9 209.85.250.175 False
10 142.250.203.195 True
11 142.250.203.195 True


In [10]:
a = traceroute(["www.google.co.uk", "www.secdev.org"], verbose=0)
a

(<Traceroute: TCP:44 UDP:0 ICMP:14 Other:0>,
 <Unanswered: TCP:2 UDP:0 ICMP:0 Other:0>)